# Part 2. Modelling the behavioral data

In Part 1, we had a look at options for preprocessing fMRI data. Now, let's turn to the other essential ingredient of model-based fMRI-analyses: modelling behavioral data.

There are Python packages that allows you to easily fit decision-making models such as the drift diffusion model (DDM). Most notably, [`hddm`](http://ski.clps.brown.edu/hddm_docs/) developed at Brown University is a widely used package.

However, `DMC` (as used in the tutorials earlier this week) has some advantages\*. For example, `DMC` offers much more models (including a multitude of LBA-model variants, stop-signal models and ex-Gaus). So you might want to use `DMC` to fit a model, and then use `Nipype` to create your fMRI-processing pipeline.

<sub><sup>\* and disadvantages - it doesn't support modelling single-trial regressors out of the box at the moment of writing</sup></sub>

So, we're just going to use `DMC` to fit the behavioral data for now. Note that _this_ notebook you're looking at right now (`Part 2. Modelling the behavioral data`) is actually _not_ an interactive Python notebook ('.ipynb'), but it is an interactive **R** notebook ('.irnb')! That means that instead of running Python code, we can run R-code in here!

For example, to draw some random numbers from a normal distribution (4,1), we can use this R-function:

In [ ]:
rnorm(10, 4, 1)

# note further that the help-function works as usual:
?rnorm

# and plotting is done in-line
plot(1:10, 1:10, main='In line plot')

### So let's use DMC to fit the behavioral data

First, set-up some directories, so R knows where to find DMC, the data, and where to store the results of these analyses (the 'res(ults)Dir')


In [ ]:
projDir <- '/home/neuro/nipype_tutorial'
dataDir <- file.path('/data', 'bids')
resDir <- file.path('/data', 'behavior_fits')
dir.create(resDir, showWarnings=FALSE)
dmcDir <- 'DMC-MBN18'

# Load dmc, load the DDM model
setwd(dmcDir)
source('dmc/dmc.R')
source('dmc/models/DDM/ddm.R')
library(rtdists)
setwd(projDir)

##### Load the data, and reformat to a DMC-approved form

In [ ]:
# Load data
dat <- read.csv(file.path(dataDir, 'behavior.tsv'), sep='\t')

# Remove unnecessary columns, rename according to DMC's wishes
dat <- dat[,c('pp', 'stimulus', 'cond', 'response', 'RT')]
colnames(dat) <- c('s', 'S', 'cue', 'R', 'RT')

# Remove all extremely fast responses (<.15s)
dat <- dat[dat$RT>.15,]

# Ensure the following columns are of type factor, and rename the stimulus/response factors to S1/S2 and R1/R2
dat$s <- factor(dat$s)
dat$cue <- factor(dat$cue, levels=c('spd', 'acc'))
levels(dat$S) <- c('S1', 'S2')
levels(dat$R) <- c('R1', 'R2')

###### The next step is to set up the model specification

In [ ]:
# set-up model: specify what 'factors' are in the experiment
factors <- list(S=c('S1', 'S2'), cue=c('spd', 'acc'))
responses <- c("R1", "R2")
match.map = list(M=list(S1="R1", S2="R2"))  # Specify which response is 'correct' for which stimulus

# set-up p.map. This tells DMC which parameters should vary across which conditions. 
# Note that we're allowing v, a, and t0 to vary across cue-condition
p.map <- list(a='cue',
              v='cue',
              t0='cue',
              z='1',
              sz='1',
              sv='1',
              st0='1',
              d='1')

# Let's not estimate between-trial variabilities. Further, fix z to 0.5 (= 0.5 * a)
constants <- c(st0=0,
               sz=0,
               sv=0,
               d=0,
               z=0.5)

# And this is our model
model <- model.dmc(constants=constants,
                   match.map=match.map,
                   factors=factors,
                   responses=responses,
                   p.map = p.map,
                   type="norm")

# combine model and data
data.model <- data.model.dmc(data=dat, model=model)

That looks good! Two thresholds (one for speed, one for accuracy), a drift rate, and a non-decision time. 

##### Now, we need to set-up priors for these parameters.

Let's use some mildly informed priors (cf. Matzke & Wagenmakers, 2009)

- v ~ TN(2.5, 3)  truncated lower at 0
- a ~ TN(1, 2)   truncated lower at 0
- t0 ~ TN(0.25, 1)  truncated lower at 0


In [ ]:
p.prior <- prior.p.dmc(
  dists = rep('tnorm', 6),
  p1=c(v.spd=2.5, v.acc=2.5, a.spd=1.00, a.acc=1.00, t0.spd=0.25, t0.acc=0.25), # these are the means
  p2=c(v.spd=3.0, v.acc=2.5, a.spd=2.00, a.acc=2.00, t0.spd=0.25, t0.acc=0.25),  # and these the SDs
  lower=c(0, 0, 0, 0, 0, 0),  # lower bound is 0 for v, a, t0
  upper=c(NA, NA, NA, NA, NA, NA)  # no upper bound
)

# What do they look like?
par(mfrow=c(3,2))
for(i in 1:length(p.prior)) plot.prior(i, p.prior)

Seems about right. Now, since fMRI sessions generally consist of very few trials (40 per speed/accuracy condition), we want to estimate these parameters in a hierarchical manner. That means that we are using group-level distributions to inform where an individual subject's parameter is likely to be.

DMC handles most of the 'magic' of hierarchical fitting. The only thing you need to do is provide priors on what you think the group-level distribution of each parameter looks like. This is typically a bit hard to do. But let's use the following priors:

In [ ]:
# Usually, I use similar priors for mu as for the subject-level, and Gamma(1,1) for sigma
mu.prior <- prior.p.dmc(
  dists = rep('tnorm', 6),
  p1=c(v.spd=2.5, v.acc=2.5, a.spd=1.00, a.acc=1.00, t0.spd=0.25, t0.acc=0.25),
  p2=c(v.spd=3.0, v.acc=2.5, a.spd=2.00, a.acc=2.00, t0.spd=0.25, t0.acc=0.25),
  lower=c(0, 0, 0, 0, 0, 0),
  upper=c(NA, NA, NA, NA, NA, NA)
)
sigma.prior <- prior.p.dmc(
  dists = rep('gamma', 6),
  p1 = c(v.spd=1, v.acc=1, a.spd=1, a.acc=1, t0.spd=1, t0.acc=1),
  p2 = c(v.spd=1, v.acc=1, a.spd=1, a.acc=1, t0.spd=1, t0.acc=1),
  lower = rep(NA, 6),
  upper = rep(NA, 6)
)
# Combine these in a list for DMC
pp.prior <- list(mu.prior, sigma.prior)

#### Now we're ready to start sampling!

In [ ]:
hsamples0 <- h.samples.dmc(nmc=250, p.prior, data.model, thin=1, pp.prior=pp.prior)

# start with a burn-in of 250 samples with migration turned on both at the subject and hyper level
hsamples1 <- h.run.dmc(hsamples0, cores=3, p.migrate=0.05, h.p.migrate=0.05, report=1)  # hsamples1 = burn-in
# This takes a while...

# Nothing stuck?
h.pick.stuck.dmc(hsamples1, start=200)
par(mfrow=c(2,2))
for(i in 1:length(hsamples1)) {
    plot.dmc(hsamples1, pll.chain=TRUE, subject = i, start=50)
    title(paste0('Subject ', names(hsamples1)[i]))
}

When I ran this before, there were no stuck chains and 250 burn-in iterations was sufficient. However, if you find stuck chains, you may want to add some extra burn-in samples before you continue.

Otherwise, let's sample from the posterior now

In [ ]:
# Let's get 1000 samples from posterior
hsamples2 <- h.run.dmc(h.samples.dmc(samples = hsamples1, nmc=1000, add=FALSE),  # don't add to burn-in but start counting from 0
                       cores=3, p.migrate=0.00, h.p.migrate=0.00, report=1)  # turn off migration in real sampling

# Let's save these results
save(hsamples0, hsamples1, hsamples2, file=file.path(resDir, 'samples.Rdata'))

##### Inspection
Now that the sampling is done, we need to inspect whether everything went right. MCMC is a bit error prone - chains can get stuck, for example. If sampling did not go well, you *cannot* interpret the resulting parameters. One thing you want to do is inspect Gelman's diagnostic

In [ ]:
gmdiag = h.gelman.diag.dmc(hsamples2)

Generally, you want them to be smaller than 1.1 (at most); I usually prefer values of 1.05 and below.

What about the effective sample sizes? Did we sample sufficient (independent) samples from the posteriors?

In [ ]:
es = effectiveSize.dmc(hsamples2)
do.call(rbind, es)  # this prints a data frame with all effective sample sizes per participant/parameter combination
min(do.call(rbind, es))  # prints the smallers number in the dataframe

\>500 samples is fine for now

It's also important to visualize and inspect your chains. First, plot the chains at the subject level

In [ ]:
options(warn=-1)  # suppresses some warnings
for(pp in names(hsamples2)) {
    par(mfrow=c(1,1))
    plot.dmc(hsamples2, hyper=FALSE, subject=pp, pll.chain=TRUE)
    mtext(paste0('Participant ', pp, ' posterior log likelihoods of all chains'), outer=TRUE, line=-1.5)
    plot.dmc(hsamples2,hyper=FALSE,subject=pp, layout=c(4,2), density = TRUE)
    mtext(paste0('Participant ', pp), outer=TRUE, line=-1.5)
}
options(warn=1)

What you want to see are "big hairy caterpillars" (left columns), that are stable (no drift up or down) and with no stuck chains ('hairs' that dont move). In the right columns, you want smooth densities.

Let's also plot the chains of the hyper parameters

In [ ]:
options(warn=-1)  # suppresses some warnings
par(mfrow=c(1,1))
plot.dmc(hsamples2, hyper=TRUE, pll.chain=TRUE)
mtext(paste0('Hyper posterior log likelihoods of all chains'), outer=TRUE, line=-1.5)
plot.dmc(hsamples2, hyper=TRUE, layout=c(4,2), density = TRUE)
mtext(paste0('Hyper'), outer=TRUE, line=-1.5)
options(warn=1)

Alright, all the previous inspections indicate that sampling went fine. Now, what about the quality of fits? In order to visualize this, we first generate 'posterior predictives', and then plot these against the real data

In [ ]:
h.posterior.preds <- h.post.predict.dmc(hsamples2, n.post = 100, cores=4)
plot.pp.dmc(h.posterior.preds, layout=c(2,2), x.min.max = c(0, 2))

What do you think of the fit?

Finally, for model-based analyses in Part 4 of this tutorial, we need parameters. Let's save these

In [ ]:
summ <- summary.dmc(hsamples2)

MAP.per.sub <- do.call(rbind, lapply(summ, function(x) x$statistics[,1]))
write.csv(MAP.per.sub, file=file.path(resDir, 'parameters_per_subject.csv'))

##### Exercise
The above fit is (I think) pretty nice, especially considering that we have only 80 trials per participant. However, it's not necessarily the _best_ model; we could consider _not_ allowing v, a, or t0 to vary between conditions, and do a formal model comparison to see which model specification has the best trade-off between complexity and quality of fit (using, e.g., DIC).

As an optional exercise, can you figure out which model specification fits the data best?

In order to find this model, you should:

 - Change the p.map
 - Change the priors 
 
And re-run the analyses above. Don't forget to *save* the current fit (hsamples2 is most important), and *not* overwrite `/data/behavioral_data/samples.Rdata` - since you need it for model comparisons!

## Additional resources
The tutorial in `DMC` is very helpful: [DMC](https://osf.io/pbwx8/?view_only=6a269eca82a44ad3992b69c7a5781af5)

Also, check out `hddm` if you want to do this type of stuff in Python: [HDDM](http://ski.clps.brown.edu/hddm_docs/)